# Test multihead coherent state generation by pullback

Use the multihead (2-head) gates in the phase space 
to create a network that represent a coherent state,
by starting from a Gaussian state and making a pullback,
then propagating through a complex random medium

<img src="../img/coherentcomplexTikZ.png" width="900" height="210" />

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>

nonlinearxwaves@gmail.com<br>
@created 22 july 2020<br>
@version 15 may 2023<br>

In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

2023-05-14 11:08:09.044590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-14 11:08:09.044615: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

## Dimension

In [3]:
N = 10

## Build vacuum by the Gaussian state

Build the layer

In [4]:
vacuum = ps.VacuumLayer(N)

2023-05-14 11:08:13.272208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-14 11:08:13.272236: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-14 11:08:13.272261: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (x1): /proc/driver/nvidia/version does not exist
2023-05-14 11:08:13.272560: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Build the pullback layer for generating the coherent state

Target displacement vector<br>
The displacement vector is a column vector

In [5]:
dinput = 3.0*np.ones((N,1))

Symplectic operator<br>
A simple identity matrix

In [6]:
D = ps.DisplacementLayerConstant(dinput)

## Define the layer for the complex medium
The LinearConstantMultiHead generate by default a non-trainable random medium with 
the relevant random symplectic operator

In [7]:
R = ps.RandomLayer(N)

## Connect the layers
Note the layers are in the inverse order as it is a pullback neural network

Input layers

In [8]:
xin = tf.keras.layers.Input(N)

Complex random medium

In [9]:
x2, a2 = R(xin)

Linear pullback for generating the coherent state

In [10]:
x1, a1 = D(x2,a2)

Gaussian vacuum state

In [11]:
chir, chii = vacuum(x1, a1)

Build the model

In [12]:
pullback = tf.keras.Model(inputs = xin, outputs=[chir, chii]) 

## Display the symplectic operator of the random layer

Evaluate the symplectic matrix and the inverse by using the builtin function

In [13]:
M, MI = R.get_M()

Print the symplectic matrix

In [14]:
utilities.printonscreen(M)

+0.2+0.0i +0.2+0.0i +0.3+0.0i +0.1+0.0i -0.3+0.0i -0.1+0.0i -0.3+0.0i -0.5+0.0i +0.2+0.0i +0.6+0.0i 
-0.2+0.0i +0.2+0.0i -0.1+0.0i +0.3+0.0i +0.1+0.0i -0.3+0.0i +0.5+0.0i -0.3+0.0i -0.6+0.0i +0.2+0.0i 
-0.1+0.0i -0.3+0.0i +0.7+0.0i +0.5+0.0i -0.1+0.0i +0.2+0.0i -0.0+0.0i +0.3+0.0i -0.3+0.0i +0.0+0.0i 
+0.3+0.0i -0.1+0.0i -0.5+0.0i +0.7+0.0i -0.2+0.0i -0.1+0.0i -0.3+0.0i -0.0+0.0i -0.0+0.0i -0.3+0.0i 
+0.0+0.0i +0.2+0.0i -0.2+0.0i +0.2+0.0i +0.7+0.0i +0.5+0.0i -0.2+0.0i +0.1+0.0i -0.0+0.0i +0.4+0.0i 
-0.2+0.0i +0.0+0.0i -0.2+0.0i -0.2+0.0i -0.5+0.0i +0.7+0.0i -0.1+0.0i -0.2+0.0i -0.4+0.0i -0.0+0.0i 
+0.7+0.0i -0.1+0.0i -0.2+0.0i -0.1+0.0i -0.1+0.0i +0.0+0.0i +0.3+0.0i +0.4+0.0i -0.2+0.0i +0.4+0.0i 
+0.1+0.0i +0.7+0.0i +0.1+0.0i -0.2+0.0i -0.0+0.0i -0.1+0.0i -0.4+0.0i +0.3+0.0i -0.4+0.0i -0.2+0.0i 
-0.2+0.0i +0.5+0.0i -0.1+0.0i +0.3+0.0i -0.3+0.0i +0.2+0.0i +0.4+0.0i +0.4+0.0i +0.4+0.0i +0.1+0.0i 
-0.5+0.0i -0.2+0.0i -0.3+0.0i -0.1+0.0i -0.2+0.0i -0.3+0.0i -0.4+0.0i +0.4+0.0i -0.1+0.0i +

In [15]:
tf.print(M)

[[0.230782926 0.225924224 0.344488919 ... -0.460717857 0.192638025 0.55475229]
 [-0.225924224 0.230782926 -0.0713840947 ... -0.33052516 -0.55475229 0.192638025]
 [-0.0772145614 -0.294446021 0.676841199 ... 0.331759214 -0.274899364 0.000528143253]
 ...
 [0.144793615 0.694200516 0.132704735 ... 0.326622039 -0.380529255 -0.240704447]
 [-0.208992183 0.478252083 -0.0779867768 ... 0.354928046 0.449485481 0.123856097]
 [-0.478252083 -0.208992183 -0.280691713 ... 0.386057556 -0.123856097 0.449485481]]


Print the inverse of the symplectic matrix

In [16]:
utilities.printonscreen(MI)

+0.2+0.0i -0.2+0.0i -0.1+0.0i +0.3+0.0i +0.0+0.0i -0.2+0.0i +0.7+0.0i +0.1+0.0i -0.2+0.0i -0.5+0.0i 
+0.2+0.0i +0.2+0.0i -0.3+0.0i -0.1+0.0i +0.2+0.0i +0.0+0.0i -0.1+0.0i +0.7+0.0i +0.5+0.0i -0.2+0.0i 
+0.3+0.0i -0.1+0.0i +0.7+0.0i -0.5+0.0i -0.2+0.0i -0.2+0.0i -0.2+0.0i +0.1+0.0i -0.1+0.0i -0.3+0.0i 
+0.1+0.0i +0.3+0.0i +0.5+0.0i +0.7+0.0i +0.2+0.0i -0.2+0.0i -0.1+0.0i -0.2+0.0i +0.3+0.0i -0.1+0.0i 
-0.3+0.0i +0.1+0.0i -0.1+0.0i -0.2+0.0i +0.7+0.0i -0.5+0.0i -0.1+0.0i -0.0+0.0i -0.3+0.0i -0.2+0.0i 
-0.1+0.0i -0.3+0.0i +0.2+0.0i -0.1+0.0i +0.5+0.0i +0.7+0.0i +0.0+0.0i -0.1+0.0i +0.2+0.0i -0.3+0.0i 
-0.3+0.0i +0.5+0.0i -0.0+0.0i -0.3+0.0i -0.2+0.0i -0.1+0.0i +0.3+0.0i -0.4+0.0i +0.4+0.0i -0.4+0.0i 
-0.5+0.0i -0.3+0.0i +0.3+0.0i -0.0+0.0i +0.1+0.0i -0.2+0.0i +0.4+0.0i +0.3+0.0i +0.4+0.0i +0.4+0.0i 
+0.2+0.0i -0.6+0.0i -0.3+0.0i -0.0+0.0i -0.0+0.0i -0.4+0.0i -0.2+0.0i -0.4+0.0i +0.4+0.0i -0.1+0.0i 
+0.6+0.0i +0.2+0.0i +0.0+0.0i -0.3+0.0i +0.4+0.0i -0.0+0.0i +0.4+0.0i -0.2+0.0i +0.1+0.0i +

Check if the matrix is symplectic by multipliying M and its inverse IM as generated by the layer

In [17]:
utilities.printonscreen(tf.matmul(M,MI))

+1.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i 
+0.0+0.0i +1.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i 
-0.0+0.0i +0.0+0.0i +1.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
-0.0+0.0i -0.0+0.0i -0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i 
-0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i 
-0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +1.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i 
-0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +1.0+0.0i +0.0+0.0i -0.0+0.0i 
-0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i -0.0+0.0i 
+0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +

## Test the derivative of the model
Evaluate the expectation value of the displacement as derivative of the characterisc function

In [18]:
x = tf.Variable(np.zeros((1,N)), dtype=tf_real) # the derivative are evaluated at x=0
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    chir, chii = pullback(x)

Derivative of chir<br>
(must be zero)

In [19]:
utilities.printonscreen(tape.gradient(chir,x))

+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 


In [20]:
tf.print(tape.gradient(chir,x))

[[0 0 0 ... 0 0 0]]


Derivative of chii<br>
(this is a random vector)

In [21]:
utilities.printonscreen(tape.gradient(chii,x))

+1.2+0.0i -0.4+0.0i +2.6+0.0i -1.6+0.0i +4.8+0.0i -3.3+0.0i +2.8+0.0i +0.1+0.0i +4.9+0.0i -3.6+0.0i 


In [22]:
tf.print(tape.gradient(chii,x))

[[1.22221088 -0.411860943 2.60023451 ... 0.0550819635 4.86147833 -3.63549471]]


Test the conservation of the photon number by the norm of input and output displacement<br>
The total number of bosons for a coherent state is <br>
$\sum_{j=0}^{N-1} \frac{1}{2} d_j^2$

In [23]:
doutput=tape.gradient(chii,x).numpy(); print(doutput)

[[ 1.2222109  -0.41186094  2.6002345  -1.6080265   4.846274   -3.3042426
   2.7805374   0.05508196  4.8614783  -3.6354947 ]]


In [24]:
print(np.linalg.norm(doutput)**2/2)

44.99999656774207


In [25]:
print(np.linalg.norm(dinput)**2/2)

45.0


Remark: The numbers are the same within numerical precision